In [1]:
import json
import numpy as np
import sys
sys.path.append('../src')
import cpu

with open('C:/Users/David/Documents/Coding/daveNES/tests/ProcessorTests-main/nes6502/v1/75.json') as f:
    test = json.load(f)

pygame 2.3.0 (SDL 2.24.2, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
subtest = 2

In [3]:
test[subtest]['initial']['ram']

[[4100, 117], [4101, 228], [4102, 217], [228, 208], [25, 128]]

In [4]:
def init_daveNES(test):
    daveNES = cpu.MOS6502(debug = False)
    daveNES.initialise_RAM()

    # Load test program
    for val in test['initial']['ram']:
        print(f'{val[0]}, {val[1]}')
        daveNES.ram.memory[val[0]] = np.uint8(val[1])
    daveNES.r_program_counter = test['initial']['pc']
    daveNES.r_stack_pointer = test['initial']['s']
    daveNES.r_accumulator = test['initial']['a']
    daveNES.r_index_X = test['initial']['x']
    daveNES.r_index_Y = test['initial']['y']
    daveNES.value_to_status(test['initial']['p'])

    return daveNES

In [5]:
daveNES = init_daveNES(test[subtest])

4100, 117
4101, 228
4102, 217
228, 208
25, 128


In [6]:
pos = daveNES.ram.read(daveNES.r_program_counter)
value = (
    pos + daveNES.r_index_X
)  # Wrapping Add (may throw overflow exception)
daveNES.r_program_counter += 1
print(value)

addr = value
value = daveNES.ram.read(addr)

a = np.uint16(daveNES.r_accumulator)
m = np.uint16(value)
c = np.uint16(daveNES.r_status["flag_C"])

print(f'{a}, {m}, {c}')

result = a + m + c

170


In [9]:
daveNES.step_program()

0x75, ADC
PC: 0x1004, SP: 0x4f, A: 0x70, X: 0x35, Y: 0x32, [1, 1, 1, 0, 0, 1, 1, 0]
281
112, 0, 0


In [10]:
print(f'{test[subtest]["final"]["pc"]} : {daveNES.r_program_counter}')
print(f'{test[subtest]["final"]["s"]} : {daveNES.r_stack_pointer}')
print(f'{test[subtest]["final"]["a"]} : {daveNES.r_accumulator}')
print(f'{test[subtest]["final"]["x"]} : {daveNES.r_index_X}')
print(f'{test[subtest]["final"]["y"]} : {daveNES.r_index_Y}')

4102 : 4102
79 : 79
240 : 112
53 : 53
50 : 50


In [11]:
# Indirect X addressing mode
daveNES.r_program_counter = test[0]['initial']['pc']
base = daveNES.ram.read(daveNES.r_program_counter)
daveNES.r_program_counter += 1
ptr = np.uint8(base) + np.uint8(daveNES.r_index_X) # Wrapping Add (may throw overflow exception)
print(f'ptr: {ptr}')
lo = daveNES.ram.read(ptr.astype(np.uint16))
hi = daveNES.ram.read(
    (ptr + np.uint8(1)).astype(np.uint16)
)  # Wrapping Add (may throw overflow exception)
addr = np.uint16(hi) << 8 | np.uint16(lo)

print(f'addr: {addr}')

#addr = np.uint16(30951) # This seems to be the issue, we want to point to 30951 as a np.uint16
value = daveNES.ram.read(addr)

a = np.uint16(daveNES.r_accumulator)
m = np.uint16(value)
c = np.uint16(daveNES.r_status["flag_C"])

result = a + m + c

print(np.uint8(result))

ptr: 53
addr: 0
112


In [6]:
pc = test[0]['initial']['pc'] + 1
base = daveNES.ram.read(pc)
pc += 1
print(f'base: {base}')
ptr = base + np.uint8(daveNES.r_index_X)
print(f'ptr: {ptr}')
#daveNES.ram.read(base + daveNES.r_index_X + np.uint8(1))

# target_number is 30951

base: 144
ptr: 27


C:\Users\David\AppData\Local\Temp\ipykernel_16108\3472765775.py:5: RuntimeWarning: overflow encountered in ubyte_scalars
  ptr = base + np.uint8(daveNES.r_index_X)


In [9]:
daveNES.ram.read((ptr + 1)) << 8 | daveNES.ram.read((ptr))

30951

In [10]:
daveNES.ram.read(np.uint16(ptr + np.uint8(1))) &0x00FF << 8

0

In [95]:
daveNES.ram.read(np.uint16(ptr) & 0x00FF)

231

In [7]:
print(f'{test[0]["final"]["pc"]} : {daveNES.r_program_counter}')
print(f'{test[0]["final"]["s"]} : {daveNES.r_stack_pointer}')
print(f'{test[0]["final"]["a"]} : {daveNES.r_accumulator}')
print(f'{test[0]["final"]["x"]} : {daveNES.r_index_X}')
print(f'{test[0]["final"]["y"]} : {daveNES.r_index_Y}')

24475 : 24475
48 : 48
73 : 73
139 : 139
31 : 31
